리뷰 총 수 다음

    # 최신순 탭 클릭하기
    try:
        recommend_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a"))
        )
        recommend_tab_element.click()
        print("최신순 탭 클릭 완료")
        time.sleep(3)  # 탭 전환 후 대기
    except Exception as e:
        print("최신순 탭 클릭 오류:", e)
    

https://map.naver.com/p/search/%ED%95%9C%EA%B0%95%EC%88%98/place/13483118?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp

In [2]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%8C%8C%EC%8A%A4%EC%BF%A0%EC%B0%8C/place/33545753?c=15.59,0,0,0,dh')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # '리뷰' 탭 클릭하기
    review_tab_element = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
    )
    review_tab_element.click()
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
    print("리뷰 탭 클릭 완료")

        # 최신순 탭 클릭하기
    try:
        recommend_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a"))
        )
        recommend_tab_element.click()
        print("최신순 탭 클릭 완료")
        time.sleep(3)  # 탭 전환 후 대기
    except Exception as e:
        print("최신순 탭 클릭 오류:", e)

    review_count = 0
    reviews = []
    max_retries = 5  # 재시도 횟수 설정

    while review_count < 1000:  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < max_retries:  # 최대 재시도 횟수
                try:
                    # 요소가 로드될 때까지 대기
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]'))
                    )

                    # 매번 요소를 새로 참조하고 스크롤을 조금씩 조정
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    # 스크롤을 조정하여 요소가 화면에 나타나도록 함
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.5)  # 스크롤 후 충분한 지연을 줌

                    # 사진이 있을 때와 없을 때의 리뷰 텍스트와 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    # 사진이 있을 때
                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        # 사진이 없을 때
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    if review_text:
                        reviews.append((review_date, review_text))
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"리뷰 {review_count}: 내용 없음")
                        review_count += 1
                    break  # 성공하면 루프 탈출
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{max_retries}")
                    time.sleep(1)  # 재시도 전 충분한 지연

            if retry_count == max_retries:
                print(f"리뷰 {i}번은 로드할 수 없어 건너뜁니다.")
                continue

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollBy(0, 300);")  # 조금씩 스크롤
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_최신순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)


리뷰 탭 클릭 완료
최신순 탭 클릭 완료
리뷰 1: 맛있는 음로가 많아요 (2024년 9월 13일 금요일)

리뷰 2: 할인도 해주고 맛난 곳 (2024년 9월 5일 목요일)

리뷰 3: 좋아요 (2024년 8월 9일 금요일)

리뷰 4: 2층 매장이 큰편은 아닙니다.
주변 회사원들이 점심시간 이후에 많이 있는것 같네요. (2024년 8월 8일 목요일)

리뷰 5: 굿 (2024년 6월 13일 목요일)

리뷰 6: 커피가 고소하고 진한데 쓰지않고 맛있어요 (2024년 6월 7일 금요일)

리뷰 7: 잠시 있을 곳이 필요해서 방문했는데 2층이 넓고 화장실도 가까워 좋습니다. 아르바이트 하시는 분들 친절하시구요. 콘센트 자리도 몇 개 있고요. 흡연실이 어마어마하게 크게 있는 것이 의외였습니다. 거의 공간의 절반을 차지했는데요. 직장인이 많아 보여서, 직장인들의 니즈를 충족시키려는 것인가 했습니다. 비흡연자는 문에서 멀리 떨어져 앉아야 해요. 문 틈사이로 희미하게 담배 냄새가 풍겨서요. 빵 종류는 매우 적고, 카운터에 있는 티세트 박스나 선물 세트 등에도 먼지가 쌓여 있어서 좀 방치되는 것 같긴 했습니다. (2024년 5월 7일 화요일)

리뷰 8: 2층매장 넓어서 쾌적하고 대화나누기 좋아요.
커피맛 괜찮아요 (2024년 4월 12일 금요일)

리뷰 9: 좋아요! (2024년 4월 4일 목요일)

리뷰 10: 파스쿠찌 커피는 사랑입니다. ^^ (2024년 3월 26일 화요일)

하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
리뷰 11: 좋아요 (2024년 3월 13일 수요일)

리뷰 12: 친절해요 (2024년 1월 25일 목요일)

리뷰 13: 적당히 달콤하고 시원하고 맛있어요 (2024년 1

KeyboardInterrupt: 

In [3]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")  # 해상도를 높여서 축소 효과를 낸다.
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%8C%8C%EC%8A%A4%EC%BF%A0%EC%B0%8C/place/33545753?c=15.59,0,0,0,dh')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)


    # 최신순 탭 클릭하기
    try:
        recommend_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a"))
        )
        recommend_tab_element.click()
        print("최신순 탭 클릭 완료")
        time.sleep(3)  # 탭 전환 후 대기
    except Exception as e:
        print("최신순 탭 클릭 오류:", e)


    review_count = 0
    reviews = []

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:  # 최대 3번 재시도
                try:
                    # 리뷰 리스트를 매번 새로 가져옴
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    # 스크롤을 조정하여 요소가 화면에 나타나도록 함
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)  # 스크롤 후 약간의 지연을 줌

                    # 사진이 있을 때와 없을 때의 리뷰 텍스트와 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    # 사진이 있을 때
                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        # 사진이 없을 때
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    if review_text:
                        reviews.append((review_date, review_text))
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"리뷰 {review_count}: 내용 없음")
                        review_count += 1
                    break  # 성공하면 루프 탈출
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)  # 재시도 전 약간의 지연

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollBy(0, 300);")  # 조금씩 스크롤
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_최신순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)


오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x0037EBD3+24307]
	(No symbol) [0x00308D74]
	(No symbol) [0x001EC323]
	(No symbol) [0x001CE00B]
	(No symbol) [0x00255F6F]
	(No symbol) [0x00268EC9]
	(No symbol) [0x0024FC26]
	(No symbol) [0x0022219C]
	(No symbol) [0x0022311D]
	GetHandleVerifier [0x00628D93+2818227]
	GetHandleVerifier [0x0068542E+3196750]
	GetHandleVerifier [0x0067D9D2+3165426]
	GetHandleVerifier [0x0041DA70+675216]
	(No symbol) [0x00311B3D]
	(No symbol) [0x0030EA18]
	(No symbol) [0x0030EBB5]
	(No symbol) [0x00301640]
	BaseThreadInitThunk [0x76C5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76F3809E+286]
	RtlGetAppContainerNamedObjectPath [0x76F3806E+238]



In [6]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%95%9C%EA%B0%95%EC%88%98/place/13483118?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    # 최신순 탭 클릭하기
    try:
        recommend_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a"))
        )
        recommend_tab_element.click()
        print("최신순 탭 클릭 완료")
        time.sleep(3)  # 탭 전환 후 대기
    except Exception as e:
        print("최신순 탭 클릭 오류:", e)

    review_count = 0
    reviews = []
    max_retries = 5  # 재시도 횟수 설정
    previous_review_count = 0  # 이전 리뷰 수 카운트
    no_new_reviews_count = 0   # 새로운 리뷰가 없는 횟수 카운트

    while review_count < 1000:  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < max_retries:  # 최대 재시도 횟수
                try:
                    # 매번 요소를 새로 참조하여 리뷰 가져오기
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    # 스크롤을 조정하여 요소가 화면에 나타나도록 함
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.5)  # 스크롤 후 충분한 지연을 줌

                    # 사진이 있을 때와 없을 때의 리뷰 텍스트와 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    # 사진이 있을 때
                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        # 사진이 없을 때
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    if review_text:
                        reviews.append((review_date, review_text))
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"리뷰 {review_count}: 내용 없음")
                        review_count += 1
                    break  # 성공하면 루프 탈출
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{max_retries}")
                    time.sleep(1)  # 재시도 전 충분한 지연

            if retry_count == max_retries:
                print(f"리뷰 {i}번은 로드할 수 없어 건너뜁니다.")
                continue

        # 새로운 리뷰가 없는 경우 종료 조건
        if review_count == previous_review_count:
            no_new_reviews_count += 1
        else:
            no_new_reviews_count = 0  # 리뷰가 추가되면 카운트를 초기화
        
        previous_review_count = review_count  # 이전 리뷰 수 갱신

        # '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollBy(0, 300);")  # 조금씩 스크롤
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")

        # 새로운 리뷰가 없는 상태가 10회 이상 지속되면 루프 종료
        if no_new_reviews_count >= 10:
            print("더 이상 새로운 리뷰가 없어 수집을 종료합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_최신순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)


가게 이름: 한강수
업종 카테고리: 정육식당
별점: 4.21
방문자 리뷰: 174
블로그 리뷰: 50
리뷰 탭 클릭 완료

리뷰 총 개수 가져오기
총 리뷰 수: 105
최신순 탭 클릭 완료
리뷰 1: 굿 (2024년 10월 29일 화요일)

리뷰 2: 모두 친철하시고, 고기질 뿐만 아니라 반찬하나하나까지 너무 맛있어요~👍🏻 (2024년 10월 12일 토요일)

리뷰 3: 동네 친구들이랑 한잔하러 왔어요
주차하기 편하고 고기 질 좋고 가성비 좋아요😊😊 (2024년 10월 8일 화요일)

리뷰 4: 굿굿 (2024년 10월 4일 금요일)

리뷰 5: 굳 (2024년 9월 19일 목요일)

리뷰 6: 굿 (2024년 9월 6일 금요일)

리뷰 7: 강남 뱅뱅매장 뒤편에 위치해있습미당~
고기집으로 퀄리티가 좋고 맛도 좋아용
점심메뉴 회덮밥 최애 메뉴네욯ㅎ
양도 많아서 아주 푸짐~
점심 시간대 피해가야 줄을 서지 않을수 있음! (2024년 7월 31일 수요일)

리뷰 8: 굿굿 (2024년 7월 19일 금요일)

리뷰 9: 굿굿 (2024년 7월 5일 금요일)

리뷰 10: 맛있게 먹었습니다~~~점심때도 손님이 많아요ㅋ (2024년 7월 3일 수요일)

하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
리뷰 11: 굿 (2024년 6월 20일 목요일)

리뷰 12: 굿굿 (2024년 5월 30일 목요일)

리뷰 13: 굿 (2024년 5월 21일 화요일)

리뷰 14: 굿굿 (2024년 5월 9일 목요일)

리뷰 15: 굿굿 (2024년 5월 2일 목요일)

리뷰 16: 강남과 양재사이 숨겨진? 가성비 맛집 (2024년 4월 26일 금요일)

리뷰 17: 맛있어요 :) (2024년 4월 19일 금요일)

리뷰 18: 굿굿 (2024년